# <b><span style="color:#C21611;">EXTRACCIÓN DE DATOS INMOBILIARIOS EN LIMA-PERÚ</span><b>

En el contexto dinámico del mercado inmobiliario en Lima, Perú, la búsqueda de información precisa y contextualizada se ha vuelto más crucial que nunca. Este proyecto, centrado en la "Extracción de Datos Inmobiliarios en Lima-Perú", no solo se propone compilar datos esenciales sobre bienes raíces, sino que aspira a proporcionar un producto final enriquecido con información adicional. La meta es desarrollar una plataforma completa que no solo proporcione información acerca de precios y ubicaciones, sino que también incluya datos sobre el entorno circundante, tales como la cantidad de parques, supermercados, hospitales y otros elementos.

Para lograr esto tambien haremos uso de APIs, las cuales nos facilitarán la conexión y comunicación entre diferentes aplicaciones y servicios. Gracias a estas herramientas, podemos acceder fácilmente a funciones específicas sin tener que construir todo desde cero

<img src="img1.png" alt="imagen1" style="width:60%;">

### <span style="color:#1557D1;">PROPÓSITO</span><b>

Este proyecto está centrado en la recopilación, procesamiento e integración de datos relacionados con el mercado inmobiliario peruano con el fin de ofrecer una plataforma grande y detallada que sirva como una herramienta valiosa para diversos actores involucrados en el mercado.

### <span style="color:#1557D1;">OBJETIVOS</span><b>

- Desarrollar una plataforma valiosa: Con técnicas avanzadas como web scraping y preprocesamiento de datos, para presentar una visión clara y objetiva del mercado inmobiliario en Lima.

- Beneficiar a múltiples partes interesadas: El proyecto no solo tiene como objetivo informar, sino también empoderar a compradores, vendedores, agentes inmobiliarios y autoridades urbanísticas con información relevante y actualizada.

- Fomentar la transparencia y toma de decisiones informada: Al proporcionar una fuente rica de datos, el proyecto busca mejorar la transparencia del mercado inmobiliario y facilitar la toma de decisiones basadas en datos sólidos.

Utilizamos la siguiente página: https://www.laencontre.com.pe/venta/casas/lima-departamento/list. 'LaEncontré' es una página donde se realizan publicaciónes de bienes inmobiliarios en el Perú, en este caso realizamos la busqueda es en la provincia de Lima, filtrando las caracteristicas más importantes, las cuales son: precio, ubicación, área, cantidad de dormitorios y baños.

### Web scraping de las primeras 5 listas

In [1]:
# Librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd

##### Obteniendo los links:

In [2]:
# URL de la página principal
url = "https://www.laencontre.com.pe/venta/casas/lima-departamento/list"

# Realiza la solicitud GET a la URL y obtiene la respuesta
response = requests.get(url)

# Utiliza BeautifulSoup para analizar el contenido HTML de la página
html = BeautifulSoup(response.content, 'html.parser')

# Encuentra todos los enlaces que terminan en p_2, p_3, p_4 y p_5
links = html.select("a.linkFilter[href$='p_2'], a.linkFilter[href$='p_3'],a.linkFilter[href$='p_4'], a.linkFilter[href$='p_5'],  a.linkFilter[href$='p_6']")

# Construye y agrega las URLs completas a una lista
base_url = "https://www.laencontre.com.pe"
complete_urls = ["https://www.laencontre.com.pe/venta/casas/lima-departamento/list"]

for link in links:
    # Obtiene el valor del atributo href de cada enlace
    href_value = link.get('href')
    
    # Construye la URL completa concatenando la base_url con el valor de href
    full_url = base_url + href_value
    
    # Agrega la URL completa a la lista
    complete_urls.append(full_url)

# Muestra la lista de URLs completas
#print(complete_urls)

### Proceso de web scraping para cada link

In [3]:
df = pd.DataFrame(columns=['Precio', 'Ubicacion', 'Área (m^2)', 'Dormitorios', 'Baños'])#, 'Descripcion'])

for link in complete_urls:
    # Realizar una solicitud HTTP a la URL actual del bucle
    page = requests.get(link)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if page.status_code == 200:
        # Analizar el contenido HTML de la página usando BeautifulSoup
        soup = BeautifulSoup(page.content, 'html.parser')

        # Obtener los datos de la página actual
        precios_list = [div.text.strip() for div in soup.find_all('div', attrs={"class": "price"})]
        ubicaciones_list = [a.text.strip() if a.text.strip() else "NA" for a in soup.find_all('a', attrs={"class": "detail-redirection"})]
        areas_list = [span.text.strip() if span.text.strip() else "NA" for span in soup.find_all('span', attrs={"class": "areaBuilt"})]
        dormitorios_list = [span.text.strip() if span.text.strip() else "NA" for span in soup.find_all('span', attrs={"class": "rooms"})]
        banos_list = [span.text.strip() if span.text.strip() else "NA" for span in soup.find_all('span', attrs={"class": "bathrooms"})]
        #descrip_list = [p.text.strip() if p.text.strip() else "NA" for p in soup.find_all('p', attrs={"class": "description"})]

        # Obtener la longitud máxima entre todas las listas
        max_length = max(len(precios_list), len(ubicaciones_list), len(areas_list), len(dormitorios_list), len(banos_list))#, len(descrip_list))

        # Llenar las listas más cortas con "NA" hasta alcanzar la longitud máxima
        precios_list += ["NA"] * (max_length - len(precios_list))
        ubicaciones_list += ["NA"] * (max_length - len(ubicaciones_list))
        areas_list += ["NA"] * (max_length - len(areas_list))
        dormitorios_list += ["NA"] * (max_length - len(dormitorios_list))
        banos_list += ["NA"] * (max_length - len(banos_list))
        #descrip_list += ["NA"] * (max_length - len(descrip_list))

        # Crear un DataFrame temporal con los datos de la página actual
        temp_df = pd.DataFrame({'Precio': precios_list,
                                'Ubicacion': ubicaciones_list,
                                'Área (m^2)': areas_list,
                                'Dormitorios': dormitorios_list,
                                'Baños': banos_list}) #,
                               # 'Descripcion': descrip_list})

        # Agregar las filas del DataFrame temporal al DataFrame principal
        df = pd.concat([df, temp_df], ignore_index=True)

    else:
        # Imprimir mensaje de error si la solicitud no fue exitosa
        print("Error al acceder a la página. Código de estado:", page.status_code)

# Imprimir mensaje de éxito
print("Datos guardados correctamente en el data frame")


Datos guardados correctamente en el data frame


In [4]:
df.head()

,Precio,Ubicacion,Área (m^2),Dormitorios,Baños
0,"$ 112,000","Casa en Unidad Vecinal Del Rimac, Lima, Perú",48m2,4,2
1,"$ 52,718.26\n \n ...","Casa en Villa Club 2, Carabayllo, Perú",76.24m2,3,2
2,"$ 44,607.76\n \n ...","Casa en Alameda De San Juan, Carabayllo, Perú",56.14m2,2,1
3,"$ 51,366.51\n \n ...","Casa en San Pedro, Carabayllo, Perú",90m2,4,2
4,(Precio a consultar)\n \n ...,"Casa en Playa Las Totoritas, Cañete, Lima, Perú",250m2,4,5


### Manipulación de datos
Vamos a transformar y preparar los datos para obtener información significativa. En esta sección, se llevará a cabo la limpieza y conversión de la algunas columnas asegurando la coherencia y calidad de los datos, preparándolos para análisis numérico y visualización.

##### Limpieza de columna 'Precio' y conversion a valores Numéricos con Tratamiento de Errores:

In [5]:
def limpiar_monto(monto):
    # Eliminar cualquier carácter no numérico al final del string
    cleaned_monto = ''.join(filter(str.isdigit, monto))
    return cleaned_monto

# Aplicar la función a la columna 'monto'
df['Precio'] = df['Precio'].apply(limpiar_monto)

In [6]:
df['Precio'] = pd.to_numeric(df['Precio'].replace('[\$,]', '', regex=True), errors='coerce')

In [7]:
print(type(df['Precio'][0]))

<class 'numpy.float64'>


##### Eliminar la subcadena "Casa en " de la columna 'Direccion'


In [8]:
df['Ubicacion'] = df['Ubicacion'].str.replace('Casa en ', '').str.strip()

In [9]:
print(df['Ubicacion'][0])

Unidad Vecinal Del Rimac, Lima, Perú


##### Conversión de 'Dormitorios' y 'Baños' a Valores Numéricos

In [10]:
df['Dormitorios'] = pd.to_numeric(df['Dormitorios'], errors='coerce').astype('Int64')
df['Baños'] = pd.to_numeric(df['Baños'], errors='coerce').astype('Int64')

In [11]:
print(type(df['Dormitorios'][0]))
print(type(df['Baños'][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>


#### Conversión de la columna Área a datos de tipo númerico

In [12]:
# Utilizar str.extract con una expresión regular para extraer el número
df['Área (m^2)'] = df['Área (m^2)'].str.extract('(\d+)')

# Llenar los valores NaN con 0 (o con el valor que prefieras)
df['Área (m^2)'] = df['Área (m^2)'].fillna(0).astype(float)

In [13]:
type(df['Área (m^2)'][0])

numpy.float64

In [36]:
df#.head()

,Precio,Ubicacion,Área (m^2),Dormitorios,Baños
0,112000.0,"Unidad Vecinal Del Rimac, Lima, Perú",48,4.0,2.0
1,5271826.0,"Villa Club 2, Carabayllo, Perú",76,3.0,2.0
2,4460776.0,"Alameda De San Juan, Carabayllo, Perú",56,2.0,1.0
3,5136651.0,"San Pedro, Carabayllo, Perú",90,4.0,2.0
4,NaN,"Playa Las Totoritas, Cañete, Lima, Perú",250,4.0,5.0
...,...,...,...,...,...
175,1090000.0,"Calle José Dionisio Anchorena, San Isidro, Perú",600,4.0,5.0
176,3800000.0,"Calle Carlos Graña, San Isidro, Perú",306,4.0,3.0
177,1850000.0,"Calle Arica, Miraflores, Perú",463,4.0,1.0
178,524000.0,"Calle General La Fuente 165, San Isidro, Perú",0,NaN,NaN


### Guardar el DataFrame en un archivo Excel y CSV

In [15]:
df.to_excel('datos_inmobiliarios.xlsx', index=False)
df.to_csv("datos_inmobiliarios.csv", index=False, encoding='utf-8')

### Creación de una tabla dinámica (Librerias Pandas y PivotTableJS)

In [ ]:
# Librería
# import pandas as pd

In [21]:
pip install pivottablejs

Note: you may need to restart the kernel to use updated packages.


In [22]:
# Libreria
from pivottablejs import pivot_ui

In [37]:
# Leer el archivo Excel
df_1= pd.read_excel('datos_inmobiliarios.xlsx', sheet_name ='Sheet1')
#eliminar filas con valores nulos
df_1_clean = df_1.dropna()

In [38]:
# Crear la tabla dinámica
pivot_ui(df_1_clean)

180